#### create sketches of different sizes for further testing

In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import *
import time
import itertools
import random
#np.random.seed([3, 2332])
#random.seed(2332)
from fbpca import pca
from geosketch import gs

base_dir = "/Users/ruthdannenfelser/Desktop/allRemote/BioGPS"

# read in the subset for testing
can = "Uveal Melanoma"
mat = pd.read_csv(base_dir + "/scripts/clustering-method/" + can.lower().replace(" ", "-") + "-test-rnaseqmat-subset.txt", sep="\t")
mat["cluster"] = 2
mat.loc[mat['tissue.cancer'] == can, "cluster"] = 1

N_samples = len(mat["tissue.cancer"])

print "original"
print mat['tissue.cancer'].value_counts()

print "everything read in"

# calculate the geometric sketch
# input matrix
matt = mat.copy()
matt.drop("batch", axis=1, inplace=True)
matt.drop("type", axis=1, inplace=True)
matt.drop("cluster", axis=1, inplace=True)
matt.drop("tissue.cancer", axis=1, inplace=True)
matt.set_index('dataset', inplace=True)
mattm = matt.values

print mattm.shape 

# compute the PCs - necessary input for the sketching
U, s, Vt = pca(mattm, k=100) # E.g.,PCs.
X_dimred = U[:, :100] * s[:100]

print "PC calculated: " + str(X_dimred.shape)

# sketchhhhhhhh
sketchsizes = [0.05, 0.1, 0.15, 0.4, 0.6, 0.8]

for ss in sketchsizes:
    print "sketch " + str(ss)
    N = int(N_samples * ss) # Number of samples to obtain from the data set 20%
    sketch_index = gs(X_dimred, N, replace=False)
    X_sketch = X_dimred[sketch_index]

    print "sketch constructed: " + str(X_sketch.shape)

    # get the samples selected in the sketch
    reduced = pd.DataFrame(X_sketch)
    pca_out = pd.DataFrame(X_dimred)
    pca_out["dataset"] = list(matt.index)

    red_with_labs = pd.merge(pca_out, reduced, how="inner", on=list(reduced.columns.values))
    selected = list(red_with_labs["dataset"])
    out = file(base_dir + "/scripts/clustering-method/sketches/" + can.lower().replace(" ", "-") + "-sketch-" + str(ss) + ".txt", "w")
    
    for elm in selected:
        out.write(elm + "\n")
    out.close()
    

original
Brain              1676
Skin               1204
Esophagus          1030
Blood Vessel        913
Adipose Tissue      797
Heart               600
Muscle              564
Colon               558
Blood               537
Lung                535
Thyroid             504
Nerve               414
Breast              402
Stomach             292
Testis              259
Pancreas            252
Liver               225
Prostate            204
Adrenal Gland       193
Pituitary           183
Kidney              173
Spleen              162
Small Intestine     137
Uterus              135
Ovary               133
Vagina              115
Salivary Gland       97
Uveal Melanoma       80
Mouth                44
Bladder              30
Cervix Uteri         14
Bile Duct             9
Fallopian Tube        7
Thymus                2
Bone                  2
Name: tissue.cancer, dtype: int64
everything read in
(12482, 2947)
PC calculated: (12482, 100)
sketch 0.05
sketch constructed: (624, 100)
sketch 0.1
sk

In [3]:
# read in large mat
everything = pd.read_csv(base_dir + "/scripts/clustering-method/rnaseq-mat-combat-zeroadj.txt", sep="\t")

In [4]:
everything.head()

,dataset,OR4F5,OR4F29,TNFRSF18,TNFRSF4,SDF4,FAM132A,SCNN1D,TAS1R3,MXRA8,...,FAM3A,CTAG1B,CTAG2,VAMP7,IL9R,PCDH11Y,NLGN4Y,batch,type,tissue.cancer
0,GTEX-1117F-0226-SM-5GZZ7,-5.0,-5.000000,2.048879,3.177841,6.784356,4.379199,2.898718,1.159374,8.034998,...,5.501882,-5.000000,-5.000000,5.533402,-0.159179,-5.0,-2.688163,gtex,tissue,Adipose Tissue
1,GTEX-1117F-0426-SM-5EGHI,-5.0,-2.633017,-0.146124,2.057661,6.023050,2.185839,1.185637,0.662423,4.211342,...,4.761571,-5.000000,-5.000000,4.750626,-5.000000,-5.0,-5.000000,gtex,tissue,Muscle
2,GTEX-1117F-0526-SM-5EGHJ,-5.0,-1.397365,1.841380,2.227611,6.615084,5.376872,2.194598,0.880796,6.149683,...,5.388699,2.416969,-0.997169,5.556323,-0.458417,-5.0,-5.000000,gtex,tissue,Blood Vessel
3,GTEX-1117F-0626-SM-5N9CS,-5.0,-5.000000,1.981671,3.164881,6.747573,1.789574,2.984512,0.956388,6.832194,...,5.731097,1.155591,3.202876,5.448268,-5.000000,-5.0,-4.132434,gtex,tissue,Blood Vessel
4,GTEX-1117F-0726-SM-5GIEN,-5.0,-2.844079,1.349044,3.216862,6.119041,-0.853638,1.737700,1.148557,5.902447,...,4.898834,-5.000000,-5.000000,4.465737,-1.660713,-5.0,-3.767002,gtex,tissue,Heart


In [7]:
cans = set(everything.loc[everything["type"] == "tumor", "tissue.cancer"].tolist())
cans

{'Acute Myeloid Leukemia',
 'Adrenocortical Carcinoma',
 'Bladder Urothelial Carcinoma',
 'Brain Lower Grade Glioma',
 'Breast Invasive Carcinoma',
 'Cervical Squamous Cell Carcinoma and Endocervical Adenocarcinoma',
 'Cholangiocarcinoma',
 'Colon Adenocarcinoma',
 'Esophageal Carcinoma',
 'Glioblastoma Multiforme',
 'Head and Neck Squamous Cell Carcinoma',
 'Kidney Chromophobe',
 'Kidney Renal Clear Cell Carcinoma',
 'Kidney Renal Papillary Cell Carcinoma',
 'Liver Hepatocellular Carcinoma',
 'Lung Adenocarcinoma',
 'Lung Squamous Cell Carcinoma',
 'Lymphoid Neoplasm Diffuse Large B-cell Lymphoma',
 'Mesothelioma',
 'Ovarian Serous Cystadenocarcinoma',
 'Pancreatic Adenocarcinoma',
 'Pheochromocytoma and Paraganglioma',
 'Prostate Adenocarcinoma',
 'Rectum Adenocarcinoma',
 'Sarcoma',
 'Skin Cutaneous Melanoma',
 'Stomach Adenocarcinoma',
 'Testicular Germ Cell Tumors',
 'Thymoma',
 'Thyroid Carcinoma',
 'Uterine Carcinosarcoma',
 'Uterine Corpus Endometrial Carcinoma',
 'Uveal Melano

In [17]:
can = "Breast Invasive Carcinoma"
mat = everything.loc[(everything["tissue.cancer"] == can),]
mat = mat.append(everything.loc[(everything["type"] == "tissue"),])
mat.head()

,dataset,OR4F5,OR4F29,TNFRSF18,TNFRSF4,SDF4,FAM132A,SCNN1D,TAS1R3,MXRA8,...,FAM3A,CTAG1B,CTAG2,VAMP7,IL9R,PCDH11Y,NLGN4Y,batch,type,tissue.cancer
12466,TCGA-3C-AAAU-01A,-5.0,-0.920001,4.980198,2.612205,6.593608,0.711309,5.308911,2.053850,5.628854,...,5.125982,-5.000000,-5.0,5.268887,-1.045759,-5.000000,-5.000000,tcga,tumor,Breast Invasive Carcinoma
12467,TCGA-3C-AALI-01A,-5.0,-5.000000,3.511310,3.798205,5.719879,1.277596,3.587489,3.186723,6.137316,...,5.080168,-1.765246,-5.0,4.248617,0.102458,-5.000000,-0.981369,tcga,tumor,Breast Invasive Carcinoma
12468,TCGA-3C-AALJ-01A,-5.0,0.282636,4.426044,4.045282,6.197203,1.738319,3.746825,1.469210,6.597637,...,4.978732,-5.000000,-5.0,4.660479,0.357884,1.344043,-5.000000,tcga,tumor,Breast Invasive Carcinoma
12469,TCGA-3C-AALK-01A,-5.0,0.042861,4.266231,3.567599,6.519250,2.362704,3.814080,1.363602,7.682358,...,4.991830,-5.000000,-5.0,4.717434,-0.467001,-5.000000,-5.000000,tcga,tumor,Breast Invasive Carcinoma
12654,TCGA-4H-AAAK-01A,-5.0,-1.342651,3.689816,3.174474,6.280569,2.040083,3.667866,0.171940,7.525970,...,4.623304,-2.811262,-5.0,4.869528,-0.359531,-5.000000,-5.000000,tcga,tumor,Breast Invasive Carcinoma


In [13]:
set(mat["tissue.cancer"].tolist())

{'Adipose Tissue',
 'Adrenal Gland',
 'Bile Duct',
 'Bladder',
 'Blood',
 'Blood Vessel',
 'Bone',
 'Brain',
 'Breast',
 'Breast Invasive Carcinoma',
 'Cervix Uteri',
 'Colon',
 'Esophagus',
 'Fallopian Tube',
 'Heart',
 'Kidney',
 'Liver',
 'Lung',
 'Mouth',
 'Muscle',
 'Nerve',
 'Ovary',
 'Pancreas',
 'Pituitary',
 'Prostate',
 'Salivary Gland',
 'Skin',
 'Small Intestine',
 'Spleen',
 'Stomach',
 'Testis',
 'Thymus',
 'Thyroid',
 'Uterus',
 'Vagina'}

In [19]:
# make quick series of sketches for breast invasive carcinoma
mat["cluster"] = 2
mat.loc[mat['tissue.cancer'] == can, "cluster"] = 1

N_samples = len(mat["tissue.cancer"])

print "original"
print mat['tissue.cancer'].value_counts()

print "everything read in"

# calculate the geometric sketch
# input matrix
matt = mat.copy()
matt.drop("batch", axis=1, inplace=True)
matt.drop("type", axis=1, inplace=True)
matt.drop("cluster", axis=1, inplace=True)
matt.drop("tissue.cancer", axis=1, inplace=True)
matt.set_index('dataset', inplace=True)
mattm = matt.values

print mattm.shape 

# compute the PCs - necessary input for the sketching
U, s, Vt = pca(mattm, k=100) # E.g.,PCs.
X_dimred = U[:, :100] * s[:100]

print "PC calculated: " + str(X_dimred.shape)

# sketchhhhhhhh
#sketchsizes = [0.05, 0.1, 0.15, 0.2, 0.4, 0.6, 0.8]
sketchsizes = [0.8]

for ss in sketchsizes:
    print "sketch " + str(ss)
    N = int(N_samples * ss) # Number of samples to obtain from the data set
    sketch_index = gs(X_dimred, N, replace=False)
    X_sketch = X_dimred[sketch_index]

    print "sketch constructed: " + str(X_sketch.shape)

    # get the samples selected in the sketch
    reduced = pd.DataFrame(X_sketch)
    pca_out = pd.DataFrame(X_dimred)
    pca_out["dataset"] = list(matt.index)

    red_with_labs = pd.merge(pca_out, reduced, how="inner", on=list(reduced.columns.values))
    selected = list(red_with_labs["dataset"])
    out = file(base_dir + "/scripts/clustering-method/sketches/" + can.lower().replace(" ", "-") + "-sketch-" + str(ss) + ".txt", "w")
    
    for elm in selected:
        out.write(elm + "\n")
    out.close()

original
Brain                        1676
Skin                         1204
Breast Invasive Carcinoma    1083
Esophagus                    1030
Blood Vessel                  913
Adipose Tissue                797
Heart                         600
Muscle                        564
Colon                         558
Blood                         537
Lung                          535
Thyroid                       504
Nerve                         414
Breast                        402
Stomach                       292
Testis                        259
Pancreas                      252
Liver                         225
Prostate                      204
Adrenal Gland                 193
Pituitary                     183
Kidney                        173
Spleen                        162
Small Intestine               137
Uterus                        135
Ovary                         133
Vagina                        115
Salivary Gland                 97
Mouth                          44
Bladd

In [20]:
# iterate through the cancer types to make sketches using 10% of the data for everything
# read in the subset for testing
for can in cans:
    print (can)
    mat = everything.loc[(everything["tissue.cancer"] == can),]
    mat = mat.append(everything.loc[(everything["type"] == "tissue"),])
    
    mat["cluster"] = 2
    mat.loc[mat['tissue.cancer'] == can, "cluster"] = 1

    N_samples = len(mat["tissue.cancer"])

    print "original"
    print mat['tissue.cancer'].value_counts()

    print "everything read in"

    # calculate the geometric sketch
    # input matrix
    matt = mat.copy()
    matt.drop("batch", axis=1, inplace=True)
    matt.drop("type", axis=1, inplace=True)
    matt.drop("cluster", axis=1, inplace=True)
    matt.drop("tissue.cancer", axis=1, inplace=True)
    matt.set_index('dataset', inplace=True)
    mattm = matt.values

    print mattm.shape 

    # compute the PCs - necessary input for the sketching
    U, s, Vt = pca(mattm, k=100) # E.g.,PCs.
    X_dimred = U[:, :100] * s[:100]

    print "PC calculated: " + str(X_dimred.shape)

    # sketchhhhhhhh
    fixed_sketch_size = 0.2
    N = int(N_samples * fixed_sketch_size) # Number of samples to obtain from the dataset
    sketch_index = gs(X_dimred, N, replace=False)
    X_sketch = X_dimred[sketch_index]

    print "sketch constructed: " + str(X_sketch.shape)

    # get the samples selected in the sketch
    reduced = pd.DataFrame(X_sketch)
    pca_out = pd.DataFrame(X_dimred)
    pca_out["dataset"] = list(matt.index)

    red_with_labs = pd.merge(pca_out, reduced, how="inner", on=list(reduced.columns.values))
    selected = list(red_with_labs["dataset"])
    out = file(base_dir + "/scripts/clustering-method/sketches/" + can.lower().replace(" ", "-") + "-sketch.txt", "w")

    for elm in selected:
        out.write(elm + "\n")
    out.close()
    

Colon Adenocarcinoma
original
Brain                   1676
Skin                    1204
Esophagus               1030
Blood Vessel             913
Adipose Tissue           797
Heart                    600
Muscle                   564
Colon                    558
Blood                    537
Lung                     535
Thyroid                  504
Nerve                    414
Breast                   402
Stomach                  292
Colon Adenocarcinoma     285
Testis                   259
Pancreas                 252
Liver                    225
Prostate                 204
Adrenal Gland            193
Pituitary                183
Kidney                   173
Spleen                   162
Small Intestine          137
Uterus                   135
Ovary                    133
Vagina                   115
Salivary Gland            97
Mouth                     44
Bladder                   30
Cervix Uteri              14
Bile Duct                  9
Fallopian Tube             7
Thymus       